In [1]:
import csv, zipfile, os, sys
import pandas as pd

In [4]:
# Read the CSV file with the ids
df = pd.read_csv('data\influencer_metadata.csv')
ids_list = df['id'].tolist()

In [ ]:
print(len(ids_list))

Run to extract OCR text (last column) from zip folder with all CSV files - Extracting CSV is not working on VCL due to limted storage, stops after extracting CSV 286. The code below was run on Google Colab

In [ ]:
def extract_data_from_csvs(zip_file_path, ids_list, output_csv):
    ids_set = set(ids_list)
    csv.field_size_limit(sys.maxsize)

    with zipfile.ZipFile(zip_file_path, 'r') as z:
        with open(output_csv, 'w', newline='', encoding='utf-8') as out_file:
            writer = csv.writer(out_file)
            writer.writerow(['id', 'text'])

            for file_name in z.namelist():
                with z.open(file_name) as csvfile:
                    lines = (line.decode('utf-8').replace('\0', '') for line in csvfile)
                    reader = csv.reader(lines, delimiter='|')
                    for row in reader:
                        if row and row[0] in ids_set:
                            print(f"Processing ID: {row[0]}")
                            writer.writerow([row[0], row[-1]])
extract_data_from_csvs(zip_file_path, ids_list, output_csv)

In [6]:
#Sanity Check 
try:
    csv.field_size_limit(2**20)
except OverflowError:
    print("The specified limit is too large!")

id_count = 0
with open('data\ocr_texts(celeb,celebrity,influencer).csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header
    for row in reader:
        id_count += 1

print(id_count)

18951


In [3]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams, trigrams
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rolando\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rolando\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
print(len(ocr_texts))

18951


Count Vectorizer

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

doc_ids = list(ocr_texts.keys())  # Get all document IDs as a list
documents = list(ocr_texts.values())  # Get all the pre-processed text values as a list

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

doc_term_matrix = vectorizer.fit_transform(documents)

# Convert the matrix to a DataFrame
df = pd.DataFrame(doc_term_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df.insert(0, 'Document ID', doc_ids)

# Save the DataFrame to a CSV file
csv_output_path = 'data\doc_term_matrix.csv'
df.to_csv(csv_output_path, index=False)